In [ ]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2
from entropy_bounds.utils import optimization_cf
from binaryIV import simulate_deterministic_data_with_probabilistic_ate, extract_prob_dict, entropy_of_array
from entropy_bounds.entropy_bounds import compute_ate_bounds


import matplotlib.pyplot as plt
import seaborn as sns
import math
from linearmodels.iv import IV2SLS
import statsmodels.api as sm
import statsmodels.formula.api as smf


pd.options.mode.chained_assignment = None  # default='warn'
pd.options.mode.chained_assignment = None  # default='warn'


pd.set_option('display.max_columns', None)

from autobound.causalProblem import causalProblem
from autobound.DAG import DAG
import io


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
data = pd.read_pickle('binaryIV_results.pkl')

In [20]:
import pandas as pd

def compute_joint_probabilities(df):
    """
    Computes the joint probabilities for each combination of Z, X, and Y in the input DataFrame.

    Parameters:
        df (pd.DataFrame): Input DataFrame with columns ['X', 'Y', 'Z'].

    Returns:
        pd.DataFrame: DataFrame with columns ['Z', 'X', 'Y', 'prob'] representing the joint probabilities.
    """
    # Count occurrences of each combination of Z, X, Y
    joint_counts = df.groupby(['Z', 'X', 'Y']).size().reset_index(name='count')
    
    # Calculate total number of rows in the input DataFrame
    total_count = len(df)
    
    # Compute probabilities
    joint_counts['prob'] = joint_counts['count'] / total_count
    
    # Drop the count column as it's not needed in the output
    joint_probs = joint_counts.drop(columns=['count'])
    
    return joint_probs

# Example usage
input_df = pd.DataFrame({
    'X': [1, 1, 0, 0, 0],
    'Y': [0, 0, 1, 0, 0],
    'Z': [0, 1, 1, 1, 0]
})

result = compute_joint_probabilities(input_df)
print(result)

   Z  X  Y  prob
0  0  0  0   0.2
1  0  1  0   0.2
2  1  0  0   0.2
3  1  0  1   0.2
4  1  1  0   0.2


In [28]:
# Minimal working example
sim = data.iloc[0]
df = pd.DataFrame({'X': sim['X'], 'Y': sim['Y'], 'Z': sim['Z']})
joint_probs = compute_joint_probabilities(df)
joint_probs.to_csv('autobound_example.csv', index=False)

dag = DAG()
dag.from_structure("Z -> X, X -> Y, U -> X, U -> Y", unob = "U")

problem = causalProblem(dag)
problem.load_data_pandas(joint_probs)
problem.add_prob_constraints()
problem.set_ate(ind='X', dep='Y')




# problem.set_ate

C:\Users\tmari\AppData\Roaming\Python\Python312\site-packages\autobound\causalProblem.py:146: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  query = [ f'{row_data.index[j]}={int(row_data[j])}'
C:\Users\tmari\AppData\Roaming\Python\Python312\site-packages\autobound\causalProblem.py:146: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  query = [ f'{row_data.index[j]}={int(row_data[j])}'
C:\Users\tmari\AppData\Roaming\Python\Python312\site-packages\autobound\causalProblem.py:146: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (co